In [11]:
import pandas as pd
import numpy as np
import scipy as sp
import nltk

from text_authorship.ta_model import load_df, train_test_split
from text_authorship.ta_model.data_preparation import get_encoder

In [29]:
df = load_df(input())

/home/dm12h/authorship/temp/transformed_df.csv


In [30]:
df.head()

,author,book,text,text_no_punkt,lemmas,tags,tokens,counts,probs,word_avg_length,words_per_sentence,exclamation_density,question_density,comma_density,dialogue_density
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,аркадий тимофеевич аверченко\nсобрание сочинен...,deleted deleted deleted собрание сочинений в ш...,deleted deleted deleted собрание сочинение шес...,deleted deleted deleted 8_NOUN 9_NOUN 5_NUMR 1...,deleted deleted deleted собрание сочинений в ш...,242,0.000043,4.617747,3.141199,0.052632,0.000000,0.000000,0.000000
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"стремясь вырваться из этой беспросветности, он...",стремясь вырваться из этой беспросветности он ...,стремиться вырваться беспросветность обращатьс...,8_GRND 9_INFN 15_NOUN 10_VERB 13_ADJF 5_NOUN d...,"стремясь вырваться из этой беспросветности , о...",242,0.000043,4.468048,2.805954,0.028571,0.000000,0.000000,0.142857
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— разрешите сесть или продолжать мне стоя? — ш...,разрешите сесть или продолжать мне стоя шутлив...,разрешить сесть продолжать стоя шутливо спроси...,9_VERB 5_INFN 10_INFN 4_ADVB 7_ADVB 7_VERB del...,— разрешите сесть или продолжать мне стоя ? — ...,242,0.000043,4.405085,2.508922,0.020833,0.020833,0.020833,0.166667
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,а в последнем номере не пропущена даже карикат...,а в последнем номере не пропущена даже карикат...,последний номер пропустить карикатура осмеиват...,9_ADJF 6_NOUN 9_PRTS 10_NOUN 11_PRTF 5_ADJF 5_...,а в последнем номере не пропущена даже карикат...,242,0.000043,4.867958,3.206627,0.000000,0.000000,0.000000,0.043478
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,помимо постоянных авторов «сатирикона» и «ново...,помимо постоянных авторов сатирикона и нового ...,помимо постоянный автор сатирикона новый сатир...,6_PREP 10_ADJF 7_NOUN 10_NOUN 6_ADJF 10_NOUN d...,помимо постоянных авторов « сатирикона » и « н...,242,0.000043,4.693370,2.947205,0.026316,0.052632,0.052632,0.105263


## CATBOOST

In [4]:
from catboost import Pool, CatBoostClassifier

Попробуем на всем  необработанном тексте, но уменьшим количество итераций и увеличим шаг для градиентного спуска

In [32]:
scalar_features = ["word_avg_length", "words_per_sentence",
                   "exclamation_density", "question_density",
                   "comma_density", "dialogue_density"]

In [33]:
df_train, df_test, y_train, y_test = train_test_split(df, share=0.5)
encoder = get_encoder(df)

df_train = df_train[scalar_features + ["tokens", "lemmas", "tags"]]
df_test = df_test[scalar_features + ["tokens", "lemmas", "tags"]]

y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [36]:
train_pool = Pool(
    df_train, 
    y_train, 
    text_features=["tokens", "lemmas", "tags"],
    
   
)
valid_pool = Pool(
    df_test, 
    y_test,
    text_features=["tokens", "lemmas", "tags"], 
   
)

catboost_params = {
    'l2_leaf_reg': 10,
    'depth': 4,
    'iterations': 300,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'task_type': 'CPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 100
}
model = CatBoostClassifier(**catboost_params)

In [ ]:
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.6029056	test: 0.1640864	best: 0.1640864 (0)	total: 368ms	remaining: 1m 50s
100:	learn: 0.8833160	test: 0.1904665	best: 0.3485435 (4)	total: 43s	remaining: 1m 24s
